In [1]:
import pandas as pd
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output  # pip install dash (version 2.0.0 or higher)




# Create the Dash app
app = Dash(__name__)

# -- Import and clean data (importing csv into pandas)
# df = pd.read_csv("intro_bees.csv")
df = pd.read_csv("data/intro_bees.csv")

df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
print(df[:5])


     State  ANSI Affected by  Year state_code  Pct of Colonies Impacted
0  Alabama     1     Disease  2015         AL                      0.05
1  Alabama     1     Disease  2016         AL                      1.20
2  Alabama     1     Disease  2017         AL                      2.25
3  Alabama     1     Disease  2018         AL                      1.30
4  Alabama     1     Disease  2019         AL                      1.80


In [2]:
# Find all unique Diseases 
df['Affected by'].unique()

array(['Disease', 'Other', 'Pesticides', 'Pests_excl_Varroa', 'Unknown',
       'Varroa_mites'], dtype=object)

In [3]:
bee_killers = ["Disease", "Other", "Pesticides", "Pests_excl_Varroa", "Unknown", "Varroa_mites"]

In [4]:
# Find all unique States 
df['State'].unique()

array(['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'New Jersey',
       'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [5]:
state_names = ['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado',
               'Connecticut', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
               'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
               'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
               'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'New Jersey',
               'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
               'Oklahoma', 'Oregon', 'Pennsylvania', 'South Carolina',
               'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
               'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [6]:
# Set up the App layout
app.layout = html.Div([

    html.H1("Visualization of the % of Bees affected by Varroa_mites in the States  ", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2015", "value": 2015},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={}),
    dcc.Graph(id='my_bee_bar_chart', figure={}),
    
    html.H1("Diseases Affecting the Bees (2015 to 2019)", style={'text-align': 'center'}),
    
    dcc.Dropdown(id="slct_impact",
                 options=[{"label": x, "value":x} for x in bee_killers],
                 value="Pesticides",
                 multi=False,
                 style={'width': "40%"}
                 ),
    
    html.Div(id='output_container_2', children=[]),
    html.Br(),
    
    dcc.Graph(id='my_bee_line_chart', figure={})
])

In [7]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure'),
     Output(component_id='my_bee_bar_chart', component_property='figure'),
     Output(component_id='output_container_2', component_property='children'),
     Output(component_id='my_bee_line_chart', component_property='figure')
    ],
    [Input(component_id='slct_year', component_property='value'),
     Input(component_id='slct_impact', component_property='value')
    ],
    allow_duplicate=True
)


# Each argument in the function connects to one input
# Since we have 1 input: value, it's connected to 1 argument: option_slctd

def update_graph(option_slctd, option_slctd2):
    # Good ractice to print the argument 
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    # Copy the df
    dff = df.copy()
    # Choose rows with the selected year in the dropdown 
    dff = dff[dff["Year"] == option_slctd]
    # Choose bees affected only by Varroa_mites disease 
    dff = dff[dff["Affected by"] == "Varroa_mites"]

    # Plotly Express
    
    # Map
    fig = px.choropleth(
        data_frame= dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )    
    
    # Bar Chart
    fig1 = px.bar(
        data_frame=dff,
        x='State',
        y='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )
    
    
    container2 = "The disease chosen by user was: {}".format(option_slctd2)

    dff2 = df.copy()
    dff2 = dff2[dff2["Affected by"] == option_slctd2]
    #dff2 = dff2[(dff2["State"] == "Idaho") | (dff2["State"] == "New York") | (dff2["State"] == "New Mexico")]
    # Filter dff2 for all states in the state_names list
    dff2 = dff2[dff2["State"].isin(state_names)]
    
    # Line Chart
    fig2=  px.line(
        data_frame=dff2,
        x='Year',
        y='Pct of Colonies Impacted',
        color='State',
        template='plotly_dark'
    )
        
    return container, fig, fig1, container2, fig2


In [8]:
if __name__ == '__main__':
    app.run_server(port=8054, debug=True)
    
# Or type http://127.0.0.1:8054/ in chrome to get the dashboard 

2015
<class 'int'>
2015
<class 'int'>
2016
<class 'int'>
2017
<class 'int'>
2018
<class 'int'>
2015
<class 'int'>
2018
<class 'int'>
2018
<class 'int'>
2018
<class 'int'>
2018
<class 'int'>
2018
<class 'int'>
